# import and Functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import signal
from scipy.fftpack import fft
from scipy import ndimage
from scipy.io import loadmat
import os
import warnings
warnings.filterwarnings('ignore')
#annots = loadmat('C:\\Users\\llinc\\GitHub\\retina_personal\\0406\\merge\\merge_0224_HMM_RL_G2.5_5min_Q100_6.5mW.mat')
%matplotlib inline

In [2]:
## EqualState assign states with equal possibility for input array x
def EqualState(x, num_state):
    xs=np.sort(x)
    binlen=int(len(x)/num_state-0.5) #round
    edges = xs[np.arange(num_state+1)*binlen]
    xstate=np.zeros(len(x))
    for i in range(num_state):
        xstate[x>=edges[i]] = i
    xstate = xstate.astype(int)
    return xstate, edges

In [3]:
def PIfunc(r, x, v, dt, window):
    negshift=window[0] # second
    posshift=window[1] # second
    shiftlen=(posshift-negshift)/dt+1
    timeshift=np.linspace(negshift,posshift,int(shiftlen))
    bitshift=np.linspace(negshift/dt,posshift/dt,int(shiftlen),dtype = 'int16')
    Information = dict()
    Information[('BROJA_2PID','SI')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','UIx')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','UIv')]=np.zeros(len(bitshift))
    Information[('BROJA_2PID','CI')]=np.zeros(len(bitshift))
    Information[('Beer','Red')]=np.zeros(len(bitshift))
    Information[('Beer','UIx')]=np.zeros(len(bitshift))
    Information[('Beer','UIv')]=np.zeros(len(bitshift))
    Information[('Beer','Syn')]=np.zeros(len(bitshift))
    
    Information[('test','SI')]=np.zeros(len(bitshift))
    Information[('test','UIx')]=np.zeros(len(bitshift))
    Information[('test','UIv')]=np.zeros(len(bitshift))
    Information[('test','CI')]=np.zeros(len(bitshift))
    # shifted data
    # shift>0 => y shifted to positive side
    for i in range(len(bitshift)):
        xx=[]
        vv=[]
        rr=[]
        shift=bitshift[i]
        if shift>0:
            xx=x[shift:]
            vv=v[shift:]
            rr=r[:(-1*shift)]
        elif shift==0:
            xx=x
            vv=v
            rr=r
        elif shift<0:
            xx=x[:shift]
            vv=v[:shift]
            rr=r[(-1*shift):]
        #find weight of each states by 3D histogram 
        xedges = np.append(np.unique(xx),(max(xx)+1))
        vedges = np.append(np.unique(vv),(max(vv)+1))
        redges = np.append(np.unique(rr),(max(rr)+1))
        dat = np.concatenate((xx[:,np.newaxis], vv[:,np.newaxis],rr[:,np.newaxis]), axis=1)
        N, edges = np.histogramdd(dat, bins=(xedges, vedges, redges))
        #Calculate all kinds of probability and make sure the shape of them, 0 -> x, 1 -> v, 2 -> r
        px=(np.sum(N,axis=(1,2))/np.sum(N))[:, np.newaxis, np.newaxis]
        pv=(np.sum(N,axis=(0,2))/np.sum(N))[np.newaxis, :, np.newaxis]
        pr=(np.sum(N,axis=(0,1))/np.sum(N))[np.newaxis ,np.newaxis, :]
        pxv=(np.sum(N,axis=2)/np.sum(N))[:, :, np.newaxis]
        pxr=(np.sum(N,axis=1)/np.sum(N))[:, np.newaxis, :]
        pvr=(np.sum(N,axis=0)/np.sum(N))[np.newaxis, :, :]
        pxvr=(N/np.sum(N))
        
        Information[('test','UIx')][i] = np.nansum(pxvr*np.log2(pxvr*px/pxv/pxr))/dt
        Information[('test','UIv')][i] = np.nansum(pxvr*np.log2(pxvr*pv/pxv/pvr))/dt


        MIxr=np.nansum(pxr*np.log2(pxr/px/pr))/dt
        MIvr=np.nansum(pvr*np.log2(pvr/pv/pr))/dt
        MIxvR=np.nansum(pxvr*np.log2(pxvr/pxv/pr))/dt
        PI_xR = np.nansum(pxr*np.log2(pxr/px/pr), axis = (0,1))
        PI_vR = np.nansum(pvr*np.log2(pvr/pv/pr), axis = (0,1))
        R = sum(np.minimum(PI_xR, PI_vR))/dt
        Information[('Beer','Red')][i] = R
        Information[('Beer','UIx')][i] = MIxr - R
        Information[('Beer','UIv')][i] = MIvr - R
        Information[('Beer','Syn')][i] = MIxvR - MIxr - MIvr + R

    return timeshift, Information

In [4]:
def Spike_Time_Generater(rdt, dt, Garmma=1):
    rdt = rdt*Garmma
    Spike_time = []
    
    counter = 0
    post_remainer_c = 0
    p = 1
    while True:
        the_random_number = 1-np.random.rand()
        while (the_random_number < p and counter < len(rdt)):
            p *= np.exp(-rdt[counter])
            counter += 1
        if counter >= len(rdt):
            break
        remainer_c = -np.log(p/the_random_number)/rdt[counter-1]
#         if remainer_c>=1 or remainer_c<=0:
#             print('shit!')
        Spike_time.append(dt*(counter-remainer_c))
        p = np.exp(-remainer_c*rdt[counter-1])
    return Spike_time[::Garmma]

# activation function
def NL(x,theta=0):
    y = np.zeros(len(x))
    for i in range(len(x)):
        if x[i]- theta>0:
            y[i]= x[i]-theta
    return(y)

# chose file

In [5]:
# exp_folder = 'D:\\GoogleDrive\\retina\\Exps\\2020\\0503'
# G = 4.5
# annots = loadmat(exp_folder+'\\merge\\'+'merge_0224_OUsmooth_RL_G'+str(G)+'_5min_Q100_6.5mW_1Hz.mat')

# exp_folder = 'D:\\GoogleDrive\\retina\\Exps\\2020\\0729'
# Fc = 8
# annots = loadmat(exp_folder+'\\merge\\'+'merge_0727_OUsmooth_Bright_UL_DR_G4.5_5min_Q100_6.5mW_'+str(Fc)+'Hz.mat')

exp_folder = 'D:\\GoogleDrive\\retina\\Chou\'s data\\20200408'
Fc_list = [2,4,7,10]
annots_list = []
for Fc in Fc_list:
    annots_list.append(loadmat(exp_folder+'\\20200408_OU_cutoff='+str(Fc)+'_sort_unit2'))

In [6]:
dt = 0.01
sampling_rate = 20000
x_list, v_list, t_cor_list, T_list = [], [], [], []
for annots in annots_list:
    TimeStamps =np.round(np.squeeze(annots['TimeStamps']))
    TimeStamps = TimeStamps.astype(int)
    x = annots['a_data'][0, TimeStamps[0]*sampling_rate:TimeStamps[1]*sampling_rate+1]
    x = ndimage.gaussian_filter1d(x, sigma=int(sampling_rate*dt/2), mode='reflect') / dt
    x = x[::int(sampling_rate*dt)]
    x = x.astype(float)
    x = (x -np.mean(x))/np.std(x)
    T = np.arange(1,len(x)+1)*dt
    v = np.append(0,np.diff(x))/dt
    
    x_list.append(x.copy())
    v_list.append(v.copy())
    T_list.append(T.copy())
    x_cor = np.correlate(x, x, "same")
    x_cor = x_cor[:int((len(x_cor)+1)/2)+1]
    Taxis = np.flip(T[:len(x_cor)]-dt*3/2)
    t_cor_list.append(np.interp(0.5*max(x_cor),  x_cor, Taxis))

# for $\beta$

In [7]:
Lambdas =  np.arange(0.05,0.95,0.02)
betas = (1-Lambdas)/Lambdas
alpha = 100. #1/sec
K = 100.
g = 75.
phi = 1.

In [ ]:
##response
# Model one: simple estimation
window = [-1,1] # second
Ux_peaktime_list =[]
Uv_peaktime_list = []
Syn_peaktime_list = []
Rdn_peaktime_list = []
MIx_peaktime_list = []
MIv_peaktime_list = []
MIxv_peaktime_list = []

Ux_peakH_list =[]
Uv_peakH_list = []
Syn_peakH_list = []
Rdn_peakH_list = []
MIx_peakH_list = []
MIv_peakH_list = []
MIxv_peakH_list = []

ngd_Ux_peaktime_list =[]
ngd_Uv_peaktime_list = []
ngd_Syn_peaktime_list = []
ngd_Rdn_peaktime_list = []
ngd_MIx_peaktime_list = []
ngd_MIv_peaktime_list = []
ngd_MIxv_peaktime_list = []

ngd_Ux_peakH_list =[]
ngd_Uv_peakH_list = []
ngd_Syn_peakH_list = []
ngd_Rdn_peakH_list = []
ngd_MIx_peakH_list = []
ngd_MIv_peakH_list = []
ngd_MIxv_peakH_list = []
for i in range(len(annots_list)):
    Ux_peaktime =[]
    Uv_peaktime = []
    Syn_peaktime = []
    Rdn_peaktime = []
    MIx_peaktime = []
    MIv_peaktime = []
    MIxv_peaktime = []
    Ux_peakH =[]
    Uv_peakH = []
    Syn_peakH = []
    Rdn_peakH = []
    MIx_peakH = []
    MIv_peakH = []
    MIxv_peakH = []
    
    ngd_Ux_peaktime =[]
    ngd_Uv_peaktime = []
    ngd_Syn_peaktime = []
    ngd_Rdn_peaktime = []
    ngd_MIx_peaktime = []
    ngd_MIv_peaktime = []
    ngd_MIxv_peaktime = []
    ngd_Ux_peakH =[]
    ngd_Uv_peakH = []
    ngd_Syn_peakH = []
    ngd_Rdn_peakH = []
    ngd_MIx_peakH = []
    ngd_MIv_peakH = []
    ngd_MIxv_peakH = []

    x = x_list[i]
    v = v_list[i]
    for Lambda in Lambdas:
        
        beta = (1-Lambda)/Lambda
        print(Fc_list[i], Lambda, beta)
        y=np.zeros(len(T_list[i]))
        z=np.zeros(len(T_list[i]))
        for j in range(len(T_list[i])-1):
#             g= beta
            dy=dt*(-alpha*y[j]+K*(x[j]-phi*z[j]))
            dz=dt*(-beta*z[j]+g*y[j])
            y[j+1]=y[j]+dy
            z[j+1]=z[j]+dz
        #assign states
        r = NL(np.mean(y)-y, 0)
        ratio = sum(r)/T_list[i][-1]/12
        FakeSpike = Spike_Time_Generater(r/ratio, dt, 10)
        rstate, _ = np.histogram(FakeSpike, np.append(0,T_list[i]))
        #rstate = np.random.poisson(rstate)
        rstate = rstate.astype(int)
        xstate,edge = EqualState(x, 6)
        vstate,edge = EqualState(v, 6)
        #calculate information
        timeshift, Information = PIfunc(rstate, xstate, vstate, dt, window)
        Ux = Information[('Beer','UIx')]
        Uv = Information[('Beer','UIv')]
        Syner = Information[('Beer','Syn')]
        Redun = Information[('Beer','Red')]
#         ax.plot(Ux+Uv+Redun+Syner)
        #find peak
        ngd_Ux_peaktime.append( timeshift[int(np.mean(np.where(Ux == max(Ux))))])
        ngd_Uv_peaktime.append( timeshift[int(np.mean(np.where(Uv == max(Uv))))])
        ngd_Syn_peaktime.append( timeshift[int(np.mean(np.where(Syner == max(Syner))))])
        ngd_Rdn_peaktime.append( timeshift[int(np.mean(np.where(Redun == max(Redun))))])
        ngd_MIx_peaktime.append( timeshift[np.argmax(Ux+Redun)])
        ngd_MIv_peaktime.append( timeshift[np.argmax(Uv+Redun)])
        ngd_MIxv_peaktime.append( timeshift[np.argmax(Ux+Uv+Redun+Syner)])
        
        ngd_Ux_peakH.append( max(Ux))
        ngd_Uv_peakH.append( max(Uv))
        ngd_Syn_peakH.append( max(Syner))
        ngd_Rdn_peakH.append( max(Redun))
        ngd_MIx_peakH.append( max(Ux+Redun))
        ngd_MIv_peakH.append( max(Uv+Redun))
        ngd_MIxv_peakH.append( max(Ux+Uv+Redun+Syner))
        
        r1 = (1-Lambda)*x+Lambda*v
        r = NL(np.mean(r1)-r1, 0)
        ratio = sum(r)/T_list[i][-1]/12
        FakeSpike = Spike_Time_Generater(r/ratio, dt, 10)
        rstate, _ = np.histogram(FakeSpike, np.append(0,T_list[i]))
        #rstate = np.random.poisson(rstate)
        rstate = rstate.astype(int)
        xstate,edge = EqualState(x, 6)
        vstate,edge = EqualState(v, 6)
        #calculate information
        timeshift, Information = PIfunc(rstate, xstate, vstate, dt, window)
        Ux = Information[('Beer','UIx')]
        Uv = Information[('Beer','UIv')]
        Syner = Information[('Beer','Syn')]
        Redun = Information[('Beer','Red')]
        #find peak
        Ux_peaktime.append( timeshift[int(np.mean(np.where(Ux == max(Ux))))])
        Uv_peaktime.append( timeshift[int(np.mean(np.where(Uv == max(Uv))))])
        Syn_peaktime.append( timeshift[int(np.mean(np.where(Syner == max(Syner))))])
        Rdn_peaktime.append( timeshift[int(np.mean(np.where(Redun == max(Redun))))])
        MIx_peaktime.append( timeshift[np.argmax(Ux+Redun)])
        MIv_peaktime.append( timeshift[np.argmax(Uv+Redun)])
        MIxv_peaktime.append( timeshift[np.argmax(Ux+Uv+Redun+Syner)])
        
        Ux_peakH.append( max(Ux))
        Uv_peakH.append( max(Uv))
        Syn_peakH.append( max(Syner))
        Rdn_peakH.append( max(Redun))
        MIx_peakH.append( max(Ux+Redun))
        MIv_peakH.append( max(Uv+Redun))
        MIxv_peakH.append( max(Ux+Uv+Redun+Syner))
        
    Ux_peaktime_list.append(np.array(Ux_peaktime).copy())
    Uv_peaktime_list.append(np.array(Uv_peaktime).copy())
    Syn_peaktime_list.append(np.array(Syn_peaktime).copy())
    Rdn_peaktime_list.append(np.array(Rdn_peaktime).copy())
    MIx_peaktime_list.append(np.array(MIx_peaktime).copy())
    MIv_peaktime_list.append(np.array(MIv_peaktime).copy())
    MIxv_peaktime_list.append(np.array(MIxv_peaktime).copy())
    
    Ux_peakH_list.append(np.array(Ux_peakH).copy())
    Uv_peakH_list.append(np.array(Uv_peakH).copy())
    Syn_peakH_list.append(np.array(Syn_peakH).copy())
    Rdn_peakH_list.append(np.array(Rdn_peakH).copy())
    MIx_peakH_list.append(np.array(MIx_peakH).copy())
    MIv_peakH_list.append(np.array(MIv_peakH).copy())
    MIxv_peakH_list.append(np.array(MIxv_peakH).copy())
    
    ngd_Ux_peaktime_list.append(np.array(ngd_Ux_peaktime).copy())
    ngd_Uv_peaktime_list.append(np.array(ngd_Uv_peaktime).copy())
    ngd_Syn_peaktime_list.append(np.array(ngd_Syn_peaktime).copy())
    ngd_Rdn_peaktime_list.append(np.array(ngd_Rdn_peaktime).copy())
    ngd_MIx_peaktime_list.append(np.array(ngd_MIx_peaktime).copy())
    ngd_MIv_peaktime_list.append(np.array(ngd_MIv_peaktime).copy())
    ngd_MIxv_peaktime_list.append(np.array(ngd_MIxv_peaktime).copy())
    
    ngd_Ux_peakH_list.append(np.array(ngd_Ux_peakH).copy())
    ngd_Uv_peakH_list.append(np.array(ngd_Uv_peakH).copy())
    ngd_Syn_peakH_list.append(np.array(ngd_Syn_peakH).copy())
    ngd_Rdn_peakH_list.append(np.array(ngd_Rdn_peakH).copy())
    ngd_MIx_peakH_list.append(np.array(ngd_MIx_peakH).copy())
    ngd_MIv_peakH_list.append(np.array(ngd_MIv_peakH).copy())
    ngd_MIxv_peakH_list.append(np.array(ngd_MIxv_peakH).copy())

2 0.05 18.999999999999996
2 0.07 13.285714285714283
2 0.09000000000000001 10.11111111111111
2 0.11000000000000001 8.09090909090909


In [ ]:
plt.plot(y)

# Plot and Save

In [ ]:
XOsave = 0
save_folder = 'C:\\Users\\hydro_leo\\Documents\\GitHub\\python-code\\4PID_paper\\beta vs. PI\\'
# save_folder = 'C:\\Users\\llinc\\GitHub\\python code\\4PID_paper\\lambda vs. PI\\'
name = 'WF_PeakHeights_Fc='+str(Fc)

## different Fc (Peak height & time)

In [ ]:
fig_list = []
for i in range(len(annots_list)):
    fig, (ax2, ax3) = plt.subplots(nrows=1, ncols=2) 
    fig.set_size_inches(16, 4*(np.sqrt(5)-1))
    fig.suptitle(r'$F_c = $'+ str(Fc_list[i]/2)+'Hz,  '+r'$t_{corr}=$'+str(round(t_cor_list[i],3))+'s',y =0.95)
    fig_list.append(fig)
    ax2.plot(Lambdas,Ux_peakH_list[i], 'r-')
    ax2.plot(Lambdas,Uv_peakH_list[i], 'b-')
    ax2.plot(Lambdas,Syn_peakH_list[i], 'k-')
    ax2.plot(Lambdas,Rdn_peakH_list[i], 'g-')
    ax2.plot(Lambdas,ngd_Ux_peakH_list[i], 'r:')
    ax2.plot(Lambdas,ngd_Uv_peakH_list[i], 'b:')
    ax2.plot(Lambdas,ngd_Syn_peakH_list[i], 'k:')
    ax2.plot(Lambdas,ngd_Rdn_peakH_list[i], 'g:')

    ax2.set_xlabel(r'$\lambda $ ')
    ax2.set_ylabel(r'Peak Height (bit/s)')
    ax2.grid(True)
    ax2.legend((r'$U_x$', r'$U_v$', 'synergy', 'redundancy'), loc='best', prop={'size': 'large', 'family': 'monospace'})

    

    ax3.plot(Lambdas,Ux_peaktime_list[i], 'r-')
    ax3.plot(Lambdas,Uv_peaktime_list[i], 'b-')
    ax3.plot(Lambdas,Syn_peaktime_list[i], 'k-')
    ax3.plot(Lambdas,Rdn_peaktime_list[i], 'g-')
    ax3.plot(Lambdas,ngd_Ux_peaktime_list[i], 'r:')
    ax3.plot(Lambdas,ngd_Uv_peaktime_list[i], 'b:')
    ax3.plot(Lambdas,ngd_Syn_peaktime_list[i], 'k:')
    ax3.plot(Lambdas,ngd_Rdn_peaktime_list[i], 'g:')
    ax3.set_xlabel(r'$\lambda$ ')
    ax3.set_ylabel('Peak time (s) ')
    ax3.legend( (r'$U_x$', r'$U_v$', 'synergy', 'redundancy'), loc='best')
    ax3.grid(True)

In [ ]:
if XOsave:
    for i in range(len(annots_list)):
        name = 'PeakHeightsnTime_Fc='+str(Fc_list[i])
        fig_list[i].savefig(save_folder+name+'.tif')

## different Fc (Ratio)

In [ ]:
Phi_list = []
for i in range(len(annots_list)):
    Phi = ndimage.gaussian_filter1d(Uv_peakH_list[i]/Ux_peakH_list[i], sigma=1, mode='reflect')
    ngd_Phi = ndimage.gaussian_filter1d(ngd_Uv_peakH_list[i]/ngd_Ux_peakH_list[i], sigma=1, mode='reflect')
    plt.plot(Lambdas,ngd_Phi,linestyle = 'dashed')
    plt.plot(Lambdas,Phi)
plt.gcf().set_size_inches(6.6, 3.3*(np.sqrt(5)-1))
plt.xlabel(r'$\lambda$ ')
plt.ylabel('peak height ratio')
plt.grid()

In [ ]:
name = 'PeakHeightnTime vs Fc'
np.savez(save_folder+name+'.npz', Lambdas=Lambdas, Ux_peakH_list=Ux_peakH_list, Uv_peakH_list=Uv_peakH_list,
         Syn_peakH_list=Syn_peakH_list, Rdn_peakH_list=Rdn_peakH_list, Ux_peaktime_list=Ux_peaktime_list, 
         Uv_peaktime_list=Uv_peaktime_list, Syn_peaktime_list=Syn_peaktime_list, Rdn_peaktime_list=Rdn_peaktime_list,
         Fc_list=Fc_list)

In [ ]:
save_folder

In [ ]:
name

In [ ]:
for i in range(len(x_list)): 
    print(np.std(x_list[i]), np.std(v_list[i]))
#     plt.plot(x_list)
#     plt.plot(v_list)